## Manual Backpropagation Part 2: Neuron

In [1]:
%cd ..

d:\youtube-andrej-karpathy\neural-networks-zero-to-hero


<img src="https://www.google.com/url?sa=i&url=https%3A%2F%2Fcs231n.github.io%2Fconvolutional-networks%2F&psig=AOvVaw29iy0a3oqNpFrzgPGjmQGv&ust=1669735360447000&source=images&cd=vfe&ved=0CBAQjRxqFwoTCPim64iX0fsCFQAAAAAdAAAAABAE">